In [7]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import json

In [7]:
event_list = []

for i in range(1,42):
    url = "https://visitseattle.org/events/page/" + str(i)
    time.sleep(1)
    res = requests.get(url)

    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "html.parser")
        selector = "div.search-result-preview > div > h3 > a"
        a_eles = soup.select(selector)

        for a_element in a_eles[:9]:
            if 'href' in a_element.attrs:
                event_url = a_element['href']
                event_list.append(event_url)
                print(event_url)

https://visitseattle.org/events/aryana-leon/
https://visitseattle.org/events/colorworks/
https://visitseattle.org/events/dar-williams/
https://visitseattle.org/events/deb-perelman/
https://visitseattle.org/events/emanuel-brown/
https://visitseattle.org/events/james-miles/
https://visitseattle.org/events/madeline-pendleton/
https://visitseattle.org/events/october-london/
https://visitseattle.org/events/tacoma-stars-vs-empire-strykers/
https://visitseattle.org/events/dungeons-and-drag-queens/
https://visitseattle.org/events/global-rhythms-yungchen-lhamo/
https://visitseattle.org/events/harlem-globetrotters-2024-world-tour/
https://visitseattle.org/events/seattle-kraken-vs-st-louis-blues/
https://visitseattle.org/events/shelf-nunny/
https://visitseattle.org/events/super-diamond-the-neil-diamond-tribute-band/
https://visitseattle.org/events/the-disorderlies-2/
https://visitseattle.org/events/compania-nacional-de-danza/
https://visitseattle.org/events/metropolis/
https://visitseattle.org/ev

In [9]:
def scrape_event_details(url):
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        name = soup.select_one('div.medium-6.columns.event-top > h1').get_text(strip=True)
        date = soup.select_one('div.medium-6.columns.event-top > h4 > span:nth-child(1)').get_text(strip=True)
        location = soup.select_one('div.medium-6.columns.event-top > h4 > span:nth-child(2)').get_text(strip=True)
        event_type = soup.select_one('div.medium-6.columns.event-top > a:nth-child(3)').get_text(strip=True)
        region = soup.select_one('div.medium-6.columns.event-top > a:nth-child(4)').get_text(strip=True)

        return [name, date, location, event_type, region]
    else:
        return ['N/A', 'N/A', 'N/A', 'N/A', 'N/A']  # In case the page request fails

# Main code to iterate through pages and scrape event details
data = []

for page in range(1, 41):
    url = f"http://visitseattle.org/events/?page={page}"
    res = requests.get(url)

    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        event_links = soup.select('div > div.container-event > h4 > a')

        for event_link in event_links:
            event_url = event_link['href']  # Extract the href attribute for detail page URL
            event_details = scrape_event_details(event_url)
            data.append(event_details)

# Writing data to CSV
with open('events.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Date', 'Location', 'Type', 'Region'])
    writer.writerows(data)

print("Data written to events.csv")

Data written to events.csv


In [14]:
NOMINATIM_API_URL = "https://nominatim.openstreetmap.org/search"

def get_location_coordinates(location_name, location_cache):
    if location_name in location_cache:
        return location_cache[location_name]

    params = {
        "q": location_name,
        "format": "jsonv2"
    }
    res = requests.get(NOMINATIM_API_URL, params=params)
    
    if res.status_code == 200:
        data = res.json()
        if data:
            location_cache[location_name] = (data[0]['lat'], data[0]['lon'])
            return data[0]['lat'], data[0]['lon']
        else:
            location_cache[location_name] = ('Not Found', 'Not Found')
            return 'Not Found', 'Not Found'
    else:
        location_cache[location_name] = ('Not Found', 'Not Found')
        return 'Not Found', 'Not Found'

def process_locations(input_csv_path, output_csv_path):
    location_cache = {}  # Cache to store the coordinates of locations already queried

    # Read existing data
    with open(input_csv_path, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        data = list(reader)
    
    # Update data with coordinates
    for row in data:
        lat, lon = get_location_coordinates(row['Location'], location_cache)
        row['Latitude'] = lat
        row['Longitude'] = lon
        print(f"Updated {row['Location']} with Latitude: {lat}, Longitude: {lon}")
        time.sleep(0.1)  # Rate limiting for API calls

    # Write updated data to a new CSV
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        fieldnames = reader.fieldnames + ['Latitude', 'Longitude']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

    # Print success message
    print("CSV has been updated with lat and lon")

# File paths for the input and output CSV files
input_csv_path = '/Users/yomaru/Desktop/TECHIN 510/Labs/TECHIN-510-lab2/events.csv'
output_csv_path = '/Users/yomaru/Desktop/TECHIN 510/Labs/TECHIN-510-lab2/events_with_lat_lon.csv'

# Process the events and write to a new CSV file
process_locations(input_csv_path, output_csv_path)

Updated Sahalee Country Club with Latitude: 47.6348885, Longitude: -122.05701598333333
Updated Henry Art Gallery with Latitude: 47.6565246, Longitude: -122.31173279895636
Updated Majestic Bay Theatres with Latitude: Not Found, Longitude: Not Found
Updated LeMay - America's Car Museum with Latitude: 47.2362108, Longitude: -122.43044382406997
Updated ACT Theatre with Latitude: 47.6108495, Longitude: -122.3323262
Updated Seattle Children's Theatre with Latitude: 47.6204553, Longitude: -122.35225726215312
Updated Henry Art Gallery with Latitude: 47.6565246, Longitude: -122.31173279895636
Updated Tractor Tavern with Latitude: 47.6657081, Longitude: -122.38280295082915
Updated Climate Pledge Arena with Latitude: 47.6219014, Longitude: -122.353990491486
Updated Museum of History & Industry with Latitude: 47.6275114, Longitude: -122.3367506
Updated 12th Avenue Arts with Latitude: 41.70339675, Longitude: -91.5874015
Updated Funhouse with Latitude: 43.6576535, Longitude: -72.2840663
Updated Burk

In [17]:
# Function to get the grid point from latitude and longitude
def get_gridpoint(lat, lon):
    gridpoint_url = f"https://api.weather.gov/points/{lat},{lon}"
    res = requests.get(gridpoint_url)
    if res.status_code == 200:
        data = res.json()
        grid_id = data['properties']['gridId']
        grid_x = data['properties']['gridX']
        grid_y = data['properties']['gridY']
        return grid_id, grid_x, grid_y
    else:
        return None, None, None

# Function to get weather data using the grid point
def get_weather_for_location(lat, lon, weather_cache):
    cache_key = f"{lat},{lon}"
    if cache_key in weather_cache:
        return weather_cache[cache_key]

    grid_id, grid_x, grid_y = get_gridpoint(lat, lon)
    if grid_id and grid_x and grid_y:
        forecast_url = f"https://api.weather.gov/gridpoints/{grid_id}/{grid_x},{grid_y}/forecast"
        res = requests.get(forecast_url)
        if res.status_code == 200:
            forecast_data = res.json()
            # Extract daytime weather forecast
            for period in forecast_data['properties']['periods']:
                if period['isDaytime']:
                    daytime_forecast = period['shortForecast']
                    weather_cache[cache_key] = daytime_forecast
                    return daytime_forecast
            return 'Daytime weather not found'
        else:
            return 'Forecast data not found'
    else:
        return 'Gridpoint not found'

def add_weather_info(input_csv_path, output_csv_path):
    weather_cache = {}

    with open(input_csv_path, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        data = list(reader)

    for row in data:
        lat, lon = row['Latitude'], row['Longitude']
        if lat != 'Not Found' and lon != 'Not Found':
            weather = get_weather_for_location(lat, lon, weather_cache)
        else:
            weather = 'Weather data not found'
        row['Weather'] = weather
        print(f"Updated weather for location {row['Location']}: {weather}")

    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        fieldnames = reader.fieldnames + ['Weather']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

    print("CSV has been updated with weather information")

input_csv_path = '/Users/yomaru/Desktop/TECHIN 510/Labs/TECHIN-510-lab2/events_with_lat_lon.csv'
output_csv_path = '/Users/yomaru/Desktop/TECHIN 510/Labs/TECHIN-510-lab2/events_with_weather.csv'

add_weather_info(input_csv_path, output_csv_path)

Updated weather for location Sahalee Country Club: Light Rain Likely
Updated weather for location Henry Art Gallery: Light Rain Likely
Updated weather for location Majestic Bay Theatres: Weather data not found
Updated weather for location LeMay - America's Car Museum: Light Rain Likely
Updated weather for location ACT Theatre: Light Rain Likely
Updated weather for location Seattle Children's Theatre: Light Rain Likely
Updated weather for location Henry Art Gallery: Light Rain Likely
Updated weather for location Tractor Tavern: Light Rain Likely
Updated weather for location Climate Pledge Arena: Light Rain Likely
Updated weather for location Museum of History & Industry: Light Rain Likely
Updated weather for location 12th Avenue Arts: Patchy Fog then Mostly Cloudy
Updated weather for location Funhouse: Forecast data not found
Updated weather for location Burke Museum: Light Rain Likely
Updated weather for location Seattle Art Museum: Light Rain Likely
Updated weather for location The Ro